In [8]:
import numpy as np
import cv2 as cv


# initial values
brightness = 10
contrast_value = 10
angle_value = 0.0
scale = 1.0

markerSize = 2

## Upload image

In [9]:
img = cv.imread('Task-9.jpg')
harris_img = img.copy()
shi_tomasi_img = img.copy()
transformed_img = img

In [10]:
# For affine
height, width = img.shape[:2]
center = (width / 2, height / 2)
# For perpective
pointList = list()
pointList.append((0, 0))
pointList.append((img.shape[1], 0))
pointList.append((img.shape[1], img.shape[0]))
pointList.append((0, img.shape[0]))
changablePoint = 0
# For Harris
# block_size - It is the size of neighbourhood considered for corner detection.
# k_size - Aperture parameter of the Sobel derivative used.
# k_p - Harris detector free parameter in the equation.
blockSize = 2
kernelSize = 3
kParam = 0.001
# For Shi_Tomasi
amountOfCorners = 80
cornersQuality = 0.05
minimalDistance = 5

# Image transformations

In [11]:
def update():
	global transformed_img
	transformed_img = img.copy()
	change_brightness(brightness)
	change_contrast(contrast_value)
	rotation_matrix = cv.getRotationMatrix2D(center, angle_value, scale)
	transformed_img = cv.warpAffine(transformed_img, rotation_matrix, (width, height))
	# For perspective transformation, you need a 3x3 transformation matrix. Straight lines will remain straight even after the transformation. To find this transformation matrix, you need 4 points on the input image and corresponding points on the output image. Among these 4 points, 3 of them should not be collinear. Then the transformation matrix can be found by the function cv.getPerspectiveTransform. Then apply cv.warpPerspective with this 3x3 transformation matrix.
	points_for_transformation = np.float32([[0, 0], [img.shape[1], 0], [img.shape[1], img.shape[0]], [0, img.shape[0]]])
	matrix = cv.getPerspectiveTransform(
		points_for_transformation,
		np.float32(pointList))
	transformed_img = cv.warpPerspective(transformed_img, matrix, (img.shape[1], img.shape[0]), cv.INTER_CUBIC,
										 borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))
	update_Harris()
	update_Shi_Tomasi()


def change_brightness(brightness):
	global transformed_img
	if brightness != 0:
		transformed_img = cv.addWeighted(transformed_img, (255 - brightness) / 255, transformed_img, 0, brightness)


def change_contrast(contrast):
	global transformed_img
	if contrast != 0:
		alpha = 131 * (contrast + 127) / (127 * (131 - contrast))
		gamma = 127 * (1 - alpha)
		transformed_img = cv.addWeighted(transformed_img, alpha, transformed_img, 0, gamma)


def change_brightness_value(val):
	global brightness
	brightness = val
	update()


def change_contrast_value(val):
	global contrast_value
	contrast_value = val
	update()


def change_rotate_angle(val):
	global angle_value
	angle_value = val
	update()


def change_rotate_scale(val):
	global scale
	if val > 0:
		scale = val / 10.0
		update()


def change_point_x_val(val):
	global pointList
	_, old_y = pointList[changablePoint]
	pointList[changablePoint] = (val, old_y)
	update()


def change_point_y_val(val):
	global pointList
	old_x, _ = pointList[changablePoint]
	pointList[changablePoint] = (old_x, val)
	update()


In [12]:
def update_Harris():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.cornerHarris(gray, blockSize, kernelSize, kParam)
	harris_img = transformed_img.copy()
	harris_img[corners > 0.01 * corners.max()] = [0, 255, 0]
	cv.imshow('Harris', harris_img)


def change_block_size(val):
	global blockSize
	blockSize = val
	update_Harris()


def change_kernel_size(val):
	global kernelSize
	if val % 2 == 0:
		val += 1
	kernelSize = val
	update_Harris()


def change_k_value(val):
	global kParam
	if val > 0:
		kParam = val / 100
		update_Harris()

In [13]:
def update_Shi_Tomasi():
	gray = cv.cvtColor(transformed_img, cv.COLOR_BGR2GRAY)
	corners = cv.goodFeaturesToTrack(gray, amountOfCorners, cornersQuality, minimalDistance)
	shi_tomasi_img = transformed_img.copy()
	corners_int = np.int0(corners)
	for corner in corners_int:
		x, y = corner.ravel()
		cv.drawMarker(shi_tomasi_img, (x, y), (255, 0, 0), cv.MARKER_CROSS, markerSize)
	cv.imshow('Shi-Tomasi', shi_tomasi_img)


def change_corners_amount(val):
	global amountOfCorners
	if val > 0:
		amountOfCorners = val
		update_Shi_Tomasi()


def change_corners_quality(val):
	global cornersQuality
	if val > 0:
		cornersQuality = val / 100.0
		update_Shi_Tomasi()


def change_minimal_distance(val):
	global minimalDistance
	minimalDistance = val
	update_Shi_Tomasi()


def change_marker_size(val):
	global markerSize
	markerSize = val
	update_Shi_Tomasi()

In [14]:
# Change original picture
cv.namedWindow('Control')
cv.createTrackbar('Brightness', 'Control', brightness, 255, change_brightness_value)
cv.createTrackbar('Contrast', 'Control', contrast_value, 128, change_contrast_value)
cv.createTrackbar('Angle', 'Control', 0, 360, change_rotate_angle)
cv.createTrackbar('Scale', 'Control', 10, 360, change_rotate_scale)
cv.createTrackbar('X', 'Control', 0, int((img.shape[1] - 1) / 2), change_point_x_val)
cv.createTrackbar('Y', 'Control', 0, int((img.shape[0] - 1) / 2), change_point_y_val)

#Harris corners
cv.imshow('Harris', harris_img)
cv.createTrackbar('Block size', 'Harris', blockSize, 50, change_block_size)
cv.createTrackbar('Kernel size', 'Harris', kernelSize, 31, change_kernel_size)
cv.createTrackbar('K', 'Harris', 10, 100, change_k_value)

#Shi-Tomasi corners
cv.imshow('Shi-Tomasi', shi_tomasi_img)
cv.createTrackbar('Corners count', 'Shi-Tomasi', amountOfCorners, 300, change_corners_amount)
cv.createTrackbar('Quality', 'Shi-Tomasi', 5, 100, change_corners_quality)
cv.createTrackbar('Min distance', 'Shi-Tomasi', minimalDistance, 100, change_minimal_distance)
cv.createTrackbar('Marker size', 'Shi-Tomasi', markerSize, 30, change_marker_size)

cv.waitKey(0)
cv.destroyAllWindows()